# 방어적 프로그래밍
객체, 함수 또는 메서드와 같은 코드의 모든 부분을 유효하지 않은 것으로부터 스스로 보호할 수 있게 하는 것  

- 예상할 수 있는 시나리오의 오류를 처리하는 방법 (에러 핸들링 프로시저)
- 발생하지 않아야 하는 오류를 처리하는 방법 (assertion)

## 에러 핸들링
에러 핸들링 프로시저는 일반적으로 데이터 입력 확인 시 자주 사용됨  
에러 핸들링의 목적은 에러에 대해 실행을 계속할 수 있을 지 아니면 극복할 수 없는 오류여서 프로그램을 중단할지를 결정하기 위함  


**에러 처리 방법의 종류**
- 값 대체
- 에러 로깅
- 예외 처리

### 1. 값 대체
오류로 인해 잘못된 값을 생성하거나 정체가 종료될 위험이 있을 경우 결과 값을 안전한 다른 값으로 대체하는 것  
- 기본 값, 잘 알려진 상수, 초기값 등으로 대체  

제공되지 않은 데이터에 기본 값을 사용하기  
- 설정되지 않은 환경 변수의 기본 값, 설정 파일의 누락된 항목 또는 함수의 파라미터 등은 기본 값으로 동작이 가능

In [2]:
configuration = {"dbport": 5432}
print(configuration.get("dbhost", "localhost"))
print(configuration.get("dbport"))

localhost
5432


In [3]:
import os
print(os.getenv("DBHOST"))
print(os.getenv("DBPROT", 5432))

None
5432


위의 두 경우 모두 두번째 파라미터를 제공하지 않으면 함수에서 정의한 기본값인 None을 반환  
사용자 정의 함수에도 파라미터의 기본값을 직접 정의 가능
```python
    def connect_database(host="localhost", port=5432):
        logger.info("다음 정보로 데이터베이스에 접속: %s:%i", host, port)
```

### 2. 예외 처리
에러가 발생하기 쉽다는 가정으로 계속 실행하는 것보다 차라리 실행을 멈추는 것이 더 좋은 경우  
예외적인 상황을 명확하게 알려주고 원래의 비즈니스 로직에 따라 흐름을 유지하는 것  
비즈니스 로직의 일부로 go-to 문을 사용하여 예외처리를 해서는 안 됨. 호출자가 알아야 하는 실질적인 문제가 있는 경우에는 예외 발생 O  
예외는 호출자에게 잘못을 알려주는 것으로 예외는 캡슐화를 약화시키기 때문에 신중하게 사용해야 함  
함수가 너무 많은 예외를 발생시키면 호출할 때마다 발생 가능한 부작용을 염두하여 문맥을 유지해야 하므로 응집력은 약하고 너무 많은 책임을 갖고 있게 됨

#### 올바른 수준의 추상화 단계에서 예외 처리
  - 함수가 처리하는 예외는 캡슐화된 로직과 일치해야 함
  
서로 다른 수준의 추상화를 혼합하는 예제  
애플리케이션에서 디코딩한 데이터를 외부 컴포넌트에 전달하는 객체

In [5]:
class DataTransport:
    """다른 레벨에서 예외를 처리하는 객체의 예"""
    
    retry_threshold: int = 5
    retry_n_times: int = 3
        
    def __init__(self, connector):
        self._connector = connector
        self.connection = None
        
    def deliver_event(self, event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("연결 실패: %s", e)
            raise
        except ValueError as e:
            logger.error("%r 잘못된 데이터 포함: %s", event, e)
            raise
            
    def connect(self):
        for _ in range(self.retry_n_times):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                "%s: 새로운 연결 시도 %is", e, self.retry_threshold)
                time.sleep(self.retry_threshold)
            else:
                return self.connection
        raise ConnectionError(f"{self.retry_n_times} 번째 재시도 연결 실패")
        
    def send(self, data):
        return self.connection.send(data)

위의 예시에서 ValueError와 ConnectionError는 별 관계가 없음  
ConnectionError는 connect 메서드 내에서 처리되어야 함 -> 행동을 명확하게 분리 가능  
ValueError는 event의 decode 메서드에 속한 에러  

위와 같이 구현을 수정하면 deliver_event에서는 예외를 catch 할 필요 X

In [7]:
def connect_with_retry(connector, retry_n_times, retry_threshold=5):
    """
    connector의 연결을 맺는다. <retry_n_times>회 재시도
    
    연결에 성공하면 connection 객체 반환
    재시도까지 모두 실패하면 ConnectionError 발생
    
    :param connector: '.connect()' 메서드를 가진 객체
    :param retry_n_times int: ''connector.connect()''를 호출 시도하는 횟수
    :param retry_threshold int: 재시도 사이의 간격
    """
    
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: 새로운 연결 시도 %is", e, retry_threshold
            )
            time.sleep(retry_threshold)
            
    exc = ConnectionError(f"{retry_n_times} 번째 재시도 연결 실패")
    logger.exception(exc)
    raise exc

deliver_event 메서드에서 위의 함수를 호출하면 됨.  

**아래의 코드는 위의 사항을 적용한 메서드**

In [9]:
class DataTransport:
    """추상화 수준에 따른 예외 분리를 한 객체의 예제"""
    retry_threshold: int = 5
    retry_n_times: int = 3
    
    def __init__(self, connector):
        self._connector = connector
        self.connection = None
        
    def deliver_event(self, events):
        self.connection = connect_with_retry(
            self._connector, self.retry_n_times, self.retry_threshold
        )
        self.send(event)
        
    def send(self, event):
        try:
            return self.connection.send(event.decode())
        except ValueError as e:
            logger.error("%r 잘못된 데이터 포함: %s", event, e)
            raise

#### Traceback 노출 금지